In [1]:
import pandas as pd
import warnings
from matplotlib import pyplot as plt 
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from datetime import time
from scipy import signal
import obspy
from obspy.signal.detrend import polynomial
import copy
warnings.filterwarnings("ignore")
xls = pd.ExcelFile('Ex4_data1.xlsx')
df1 = pd.read_excel(xls, 'Intraday_data')
df1_copy = df1.copy(deep=True)
df2 = pd.read_excel(xls, 'Interday_data')
df2 = df2[['Date','Price']]
df2_copy = df2.copy(deep=True)
df=(df1,df2,df1_copy,df2_copy)


#linear intrapolate
for i in df:
    for n in list(i.columns):
        if (i[n].isnull().values.any()):
            i[n]=i[n].interpolate(method='linear')




In [2]:
#detrend

df1['detrend'] = signal.detrend(df1["HSI_Price"], axis=-1, type='linear', bp=0)

In [3]:
def in_timerange(check):
    if ((time(9,30)<=check) and (time(9,59)>=check)):
        return '09:30-09:59'
    elif ((time(10,0)<=check) and (time(10,29)>=check)):
        return '10:00-10:29'
    elif ((time(10,30)<=check) and (time(10,59)>=check)):
        return '10:30-10:59'
    elif ((time(11,0)<=check) and (time(11,29)>=check)):
        return '11:00-11:29'
    elif ((time(11,30)<=check) and (time(12,5)>=check)):
        return '11:30-12:05'
    elif ((time(13,0)<=check) and (time(13,29)>=check)):
        return '13:00-13:29'
    elif ((time(13,30)<=check) and (time(13,59)>=check)):
        return '13:30-13:59'
    elif ((time(14,0)<=check) and (time(14,29)>=check)):
        return '14:00-14:29'
    elif ((time(14,30)<=check) and (time(14,59)>=check)):
        return '14:30-14:59'
    elif ((time(15,0)<=check) and (time(15,29)>=check)):
        return '15:00-15:29'
    elif ((time(15,30)<=check) and (time(15,59)>=check)):
        return '15:30-15:59'

In [4]:
#create dummy variable

df1['Datetime'] = pd.to_datetime(df1['Datetime'])
df1['TimeRange'] = df1['Datetime'].dt.time.apply(in_timerange)


#create dummy variable
df1=pd.get_dummies(df1)
df1=df1.set_index('Datetime')
df1=df1.drop(['TimeRange_09:30-09:59'],axis=1)
df11 = df1.copy()
df1.drop(["HSI_Price"], axis=1,inplace=True)

In [5]:
#OLS model fitting
X = df1.drop('detrend', axis = 1)
X = sm.add_constant(X)
y = df1.loc[:,'detrend']
model=sm.OLS(y, X).fit()
print(model.summary())
print(model.rsquared)

                            OLS Regression Results                            
Dep. Variable:                detrend   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.1969
Date:                Sun, 12 Mar 2023   Prob (F-statistic):              0.997
Time:                        23:14:50   Log-Likelihood:                -44336.
No. Observations:                6035   AIC:                         8.869e+04
Df Residuals:                    6024   BIC:                         8.877e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.27

In [6]:
best_buy = df11.loc[df1['TimeRange_13:00-13:29'] == 1]['HSI_Price'].min()
best_sell = df11.loc[df1['TimeRange_15:30-15:59'] == 1]['HSI_Price'].max()

s2b = best_sell / best_buy
round(s2b,3)

1.075

In [7]:
#improve method
#calculate average price for every half hour
df1_copy['Datetime'] = pd.to_datetime(df1_copy['Datetime'])
df1_copy=df1_copy.set_index('Datetime')
df1_copy=df1_copy.resample('30T').sum()/30

#drop the useless period
df1_copy.drop(df1_copy[df1_copy.HSI_Price < 10000].index, inplace=True)

In [8]:
#detrend
df1_copy['detrend'] = signal.detrend(df1_copy["HSI_Price"], axis=-1, type='linear', bp=0)


In [9]:
#create dummy variable
df1_copy=df1_copy.reset_index()
df1_copy['Time']=df1_copy['Datetime'].dt.time
df1_copy=pd.get_dummies(df1_copy)
df1_copy=df1_copy.set_index('Datetime')
df1_copy=df1_copy.drop(['Time_09:30:00'],axis=1)
df11_copy = df1_copy.copy()
df1_copy.drop(["HSI_Price"], axis=1,inplace=True)

X = df1_copy.drop('detrend', axis = 1)
X = sm.add_constant(X)
y = df1_copy.loc[:,'detrend']
model=sm.OLS(y, X).fit()
print(model.summary())
print(model.rsquared)

                            OLS Regression Results                            
Dep. Variable:                detrend   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                    0.5559
Date:                Sun, 12 Mar 2023   Prob (F-statistic):              0.848
Time:                        23:14:50   Log-Likelihood:                -1530.5
No. Observations:                 198   AIC:                             3083.
Df Residuals:                     187   BIC:                             3119.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          -274.0050    133.512     -2.052

In [10]:
best_buy = df11_copy.loc[df1_copy.iloc[:,1:-1].T.any()==False]["HSI_Price"].min()
best_sell = df11_copy.loc[df1_copy['Time_10:00:00'] == 1]["HSI_Price"].max()

s2b = best_sell / best_buy
round(s2b,3)

1.375

# Interday Data

In [11]:
df2['detrend'] = signal.detrend(df2["Price"], axis=-1, type='linear', bp=0)


In [12]:
#day of the week
df2['Date'] = pd.to_datetime(df2['Date'])
df2['weekday'] = df2['Date'].dt.day_name()
df2['month'] = df2['Date'].dt.strftime('%b')
df2=df2.set_index('Date')
df2=pd.get_dummies(df2)

#drop not usual deal weekday
df2.drop(df2[df2['weekday_Saturday'] ==1].index, inplace=True)
df2=df2.drop(['weekday_Saturday'],axis=1)

#drop 1-hot encoding
df2=df2.drop(['weekday_Monday'],axis=1)
df2=df2.drop(['month_Jan'],axis=1)
df22=df2.copy()
df2.drop(["Price"], axis=1,inplace=True)

In [13]:
X = df2.drop('detrend', axis = 1)
X = sm.add_constant(X)
y = df2.loc[:,'detrend']
model=sm.OLS(y, X).fit()
print(model.summary())
print(model.rsquared)

                            OLS Regression Results                            
Dep. Variable:                detrend   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.651
Date:                Sun, 12 Mar 2023   Prob (F-statistic):             0.0535
Time:                        23:14:50   Log-Likelihood:                -86352.
No. Observations:                9165   AIC:                         1.727e+05
Df Residuals:                    9149   BIC:                         1.729e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               164.1104    124.23

In [14]:

day_best_buy = df22.loc[df2['weekday_Friday'] == 1]['Price'].min()
day_best_sell = df22.loc[df2['weekday_Wednesday'] == 1]['Price'].max()

day_s2b = day_best_sell / day_best_buy
round(day_s2b,3)

16.565

In [15]:
month_best_buy = df22.loc[df2['month_Oct'] == 1]['Price'].min()
month_best_sell = df22.loc[df2['month_Feb'] == 1]['Price'].max()

month_s2b = month_best_sell / month_best_buy
round(month_s2b,3)

14.807

improve method

In [16]:


#detrend
price=df2_copy[['Date','Price']]
price.set_index('Date',inplace=True)
df2_copy['detrend'] = polynomial(df2_copy['Price'], order=2)  


In [17]:
#day of the week
#calculate average price for every half hour
df2_copy['Date'] = pd.to_datetime(df2_copy['Date'])
df2_copy['weekday'] = df2_copy['Date'].dt.strftime('%A')
df2_copy['month'] = df2_copy['Date'].dt.strftime('%b')
df2_copy=pd.get_dummies(df2_copy)

#drop not usual deal weekday
df2_copy.drop(df2_copy[df2_copy['weekday_Saturday'] ==1].index, inplace=True)
df2_copy=df2_copy.drop(['weekday_Saturday'],axis=1)

#1-hot encoding
df2_copy=df2_copy.drop(['weekday_Monday'],axis=1)
df2_copy=df2_copy.drop(['month_Jan'],axis=1)

#ready s2b ratio
df2_copy = df2_copy.set_index('Date')
df22_copy=df2_copy.copy()
df2_copy.drop(['Price'], axis=1,inplace=True)

In [18]:

X = df2_copy.drop('detrend', axis = 1)
X = sm.add_constant(X)
y = df2_copy.loc[:,'detrend']
model=sm.OLS(y, X).fit()
print(model.summary())
print(model.rsquared)

                            OLS Regression Results                            
Dep. Variable:                detrend   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.859
Date:                Sun, 12 Mar 2023   Prob (F-statistic):             0.0225
Time:                        23:14:50   Log-Likelihood:                -86122.
No. Observations:                9165   AIC:                         1.723e+05
Df Residuals:                    9149   BIC:                         1.724e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               194.9839    121.15

In [19]:
df22_copy=df22.merge(price,how='left', left_index=True, right_index=True)
day_best_buy = df22_copy.loc[df2_copy['weekday_Wednesday'] == 1]['Price_x'].min()
day_best_sell = df22_copy.loc[df2_copy['weekday_Friday'] == 1]['Price_x'].max()

day_s2b = day_best_sell / day_best_buy
round(day_s2b,3)

16.702

In [20]:
month_best_buy = df22_copy.loc[df2_copy['month_Oct'] == 1]['Price_x'].min()
month_best_sell = df22_copy.loc[df2_copy['month_Feb'] == 1]['Price_x'].max()

month_s2b = month_best_sell / month_best_buy
round(month_s2b,3)

14.807